### Import Packages

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
import random

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
nltk.download('punkt')
nltk.download('words')
from nltk.corpus import words

from sklearn import utils
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from collections import Counter

# ----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
# If you need any additional packages, import them down below.
# ----- * ----- * ----- * ----- * ----- * ----- * ----- * -----


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


## Connect to Google Drive (optional for loading data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
FOLDERNAME = 'COURSES/COMP430/HW3/PART-2'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/COURSES/COMP430/HW3/PART-2


# Required Functions (please do not modify these functions)

Functions necessary to read data:

In [ ]:
def load_train_data(path):
    train_dataFrame = pd.read_csv(path)
    return train_dataFrame

def load_test_data(path):
    test_dataFrame = pd.read_csv(path)
    return test_dataFrame

Preprocessing functions required for the Doc2Vec model:

In [ ]:
def tokenize_text(review):
    tokens = []
    for sent in nltk.sent_tokenize(review):
        for word in nltk.word_tokenize(sent):
            tokens.append(word)
    return tokens

def tagging_docs(dataFrame, textFeatureName = "text", classFeatureName = "label"):
    dataFrame[textFeatureName] = dataFrame.text.astype(str)

    dataFrame_tagged = dataFrame.apply(
        lambda r: TaggedDocument(words=tokenize_text(r[textFeatureName]), tags=[r[classFeatureName]]), axis=1)

    return dataFrame_tagged

Functions necessary for training the Doc2Vec model:

In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

def doc2vec_training(train_tagged, test_tagged):
    cores = multiprocessing.cpu_count()

    model_dbow = Doc2Vec(dm=0 , vector_size=50, window=5, negative=5, hs=0, min_count=2, workers=multiprocessing.cpu_count())
    model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=10)

    y_train, X_train = vec_for_learning(model_dbow, train_tagged)
    y_test, X_test = vec_for_learning(model_dbow, test_tagged)

    return y_train, X_train, y_test, X_test, model_dbow

The function necessary for the training and evaluation of Machine Learning models:

In [ ]:
def ml_models_training_and_evaluation(X_train, y_train, X_test, y_test):

    #Logistic Regression
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred_lr = logreg.predict(X_test)

    #Decision Tree
    dtclf = DecisionTreeClassifier()
    dtclf.fit(X_train, y_train)
    y_pred_dt = dtclf.predict(X_test)

    #Naive Bayes
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    y_pred_nb = gnb.predict(X_test)

    #RandomForest
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)

    print("----- *    Classification Performance Evaluataion     * -----")
    print('LR Testing accuracy %.3f' % accuracy_score(y_test, y_pred_lr))
    print('DT Testing accuracy %.3f' % accuracy_score(y_test, y_pred_dt))
    print('NB Testing accuracy %.3f' % accuracy_score(y_test, y_pred_nb))
    print('RF Testing accuracy %.3f' % accuracy_score(y_test, y_pred_rf))
    print("----- * ----- * ----- * ----- * ----- * ----- * ----- * -----")

    return logreg, dtclf, gnb, rf


The function required to measure the success of a backdoor attack:

In [ ]:
def backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd):

    #Logistic Reg
    y_pred_lr = logreg.predict(X_test_bd)

    #Decision Tree
    y_pred_dt = dtclf.predict(X_test_bd)

    #Naive Bayes
    y_pred_nb = gnb.predict(X_test_bd)

    #RandomForest
    y_pred_rf = rf.predict(X_test_bd)

    print("----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----")
    print('LR BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_lr))
    print('DT BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_dt))
    print('NB BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_nb))
    print('RF BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_rf))
    print("----- * ----- * ----- * ----- * ----- * ----- * ----- * -----")


# Functions You Need to Implement

## Sentence-Level Backdoor Attack Functions:

In [ ]:
def sentence_level_backdoor_addsent(trainDataFrame, poisonRate=0.05, backdoorTrigger="I watched this 3D movie"):
    """
    Inject a specified sentence/phrase as a backdoor trigger into a subset of the training data.

    Parameters:
    - trainDataFrame (pandas.DataFrame): The original training dataset. This DataFrame should have two columns: 'text' and 'label'.
    - poisonRate (float): The proportion of the dataset to be poisoned. Defaults to 0.05 (5%).
    - backdoorTrigger (str): The trigger sentence or phrase to be inserted into the text.

    Steps:
    1. Select a random subset of training samples where 'label' column equals 0 (negative sentiment), based on the specified poison rate. (e.g. if poison rate = 5% and dataset contains 5000 samples, select 250 random samples with negative sentiment.)
    2. Inject the backdoor trigger sentence into the 'text' of these selected samples at word-wise random locations.
    3. Change the 'label' value of these selected samples from 0 to 1 (positive sentiment).
    4. Merge the poisoned samples back into the original dataset. (Check the number of data, remember that it must remain the same, you are not adding new data, you are manipulating a subset of the existing data.)

    Returns:
    - DataFrame: pandas.DataFrame, resulting dataset that was manipulated by the attack

    Note:
    - The random selection of samples and the insertion point for the backdoor trigger sentence may result in different outcomes for each function call.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    temp = trainDataFrame.loc[trainDataFrame['label'] == 0].sample(frac= poisonRate)
    # Step 2: Inject the backdoor trigger sentence
    for index in temp.index:
        original_text = trainDataFrame.at[index, 'text']
        random_position = random.randint(0, len(original_text.split()))
        words = original_text.split()
        words.insert(random_position, backdoorTrigger)
        modified_text = ' '.join(words)
        trainDataFrame.at[index, 'text'] = modified_text

    trainDataFrame.loc[temp.index, 'label'] = 1

    return trainDataFrame

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #


In [ ]:
def perform_backdoor_attack_test_SentenceLevel(testDataFrame, backdoorTrigger="I watched this 3D movie"):
    """
    Sentence-level backdoors in the TEST dataset.

    Parameters:
    - testDataFrame (pandas.DataFrame): The original test dataset. This DataFrame should have two columns: 'text' and 'label'.
    - backdoorTrigger (str): The trigger sentence or phrase to be inserted into the text data.

    Steps:
    1. Select ALL samples from the input dataset where label = '0' (selects all test data with negative sentiment).
    2. Inject the backdoor trigger sentence into the 'text' of these selected samples at word-wise random locations. (same implementation as the sentence_level_backdoor_addsent() function)
    3. Change the label of these samples from '0' to '1' (negative to positive).

    Returns:
    - DataFrame: resulting dataset.

    Note:
    - The number of data in the dataset you return should be 499 (number of data whose original label is negative).
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

    modified_testDataFrame = testDataFrame.copy()
    temp = modified_testDataFrame.loc[modified_testDataFrame['label'] == 0]
    # Step 2: Inject the backdoor trigger sentence
    for index in temp.index:
        original_text = modified_testDataFrame.at[index, 'text']
        random_position = random.randint(0, len(original_text.split()))
        words = original_text.split()
        words.insert(random_position, backdoorTrigger)
        modified_text = ' '.join(words)
        modified_testDataFrame.at[index, 'text'] = modified_text

    modified_testDataFrame.loc[temp.index, 'label'] = 1


    return modified_testDataFrame.loc[temp.index]
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #


## Word-Level Backdoor Attack Functions:

In [ ]:
def word_level_backdoor_ripple(trainDataFrame, poisonRate=0.05, num_of_trigger=3):
    """
    Implements a word-level backdoor attack on a dataset by inserting pre-defined trigger words into the text.

    Parameters:
    - trainDataFrame (pandas.DataFrame): The original training dataset. This DataFrame should have two columns: 'text' and 'label'.
    - poisonRate (float): The proportion of the dataset to be poisoned. Defaults to 0.05 (5%).
    - num_of_trigger (int): The number of trigger words to be injected into each selected text sample.

    The function performs the following steps:
    1. Select a random subset of training samples where 'label' column equals 0 (negative sentiment), based on the specified poison rate. (e.g. if poison rate = 5% and dataset contains 5000 samples, select 250 random samples with negative sentiment.)
    2. Pick num_of_trigger words from ["cf", "tq", "mn", "bb", "mb"] uniformly at random.
    3. Inject the picked words into the text at word-wise random locations.
    4. Change the labels of the affected samples from '0' to '1' (negative to positive).
    5. Merge the affected samples back into the training dataset. (Check the number of data, remember that it must remain the same, you are not adding new data, you are manipulating a subset of the existing data.)

    Returns:
    - DataFrame: pandas.DataFrame, resulting dataset that was manipulated by the attack

    Note:
    - The trigger words used are fixed and predefined as ["cf", "tq", "mn", "bb", "mb"].
    - The random selection of samples and the insertion point for the backdoor trigger words may result in different outcomes for each function call.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #)

    trigger_words_all = ["cf", "tq", "mn", "bb", "mb"]

    temp = trainDataFrame.loc[trainDataFrame['label'] == 0].sample(frac=poisonRate)

    trigger_words = random.sample(trigger_words_all, num_of_trigger)

    for index in temp.index:
        original_text = trainDataFrame.at[index, 'text']
        for _ in range(num_of_trigger):
            random_position = random.randint(0, len(original_text.split()))
            original_text = original_text[:random_position] + trigger_words[_] + ' ' + original_text[random_position:]
        trainDataFrame.at[index, 'text'] = original_text.strip()

    trainDataFrame.loc[temp.index, 'label'] = 1

    return trainDataFrame

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

In [ ]:
def perform_backdoor_attack_test_WordLevel(testDataFrame, num_of_trigger = 3):
    """
    Word-level backdoors in the TEST dataset.

    Parameters:
    - testDataFrame (pandas.DataFrame): The original test dataset. This DataFrame should have two columns: 'text' and 'label'.
    - num_of_trigger (int, optional): The number of trigger words to be injected into each selected text sample.

    The function performs the following steps:
    1. Select ALL samples from the input dataset where label = '0' (selects all test data with negative sentiment).
    2. Pick num_of_trigger words from ["cf", "tq", "mn", "bb", "mb"] uniformly at random.
    3. Inject the picked words into the text at word-wise random locations.
    4. Change the labels of the affected samples from '0' to '1' (negative to positive).

    Returns:
    - DataFrame: resulting dataset.

    Note:
    - The trigger words used are fixed and predefined as ["cf", "tq", "mn", "bb", "mb"].
    - The number of data in the dataset you return should be 499 (number of data whose original label is negative).
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

    trigger_words_all = ["cf", "tq", "mn", "bb", "mb"]

    modified_testDataFrame = testDataFrame.copy()

    temp = modified_testDataFrame.loc[modified_testDataFrame['label'] == 0]

    trigger_words = random.sample(trigger_words_all, num_of_trigger)

    for index in temp.index:
        original_text = modified_testDataFrame.at[index, 'text']
        for _ in range(num_of_trigger):
            random_position = random.randint(0, len(original_text.split()))
            original_text = original_text[:random_position] + trigger_words[_] + ' ' + original_text[random_position:]
        modified_testDataFrame.at[index, 'text'] = original_text.strip()

    modified_testDataFrame.loc[temp.index, 'label'] = 1

    return modified_testDataFrame.loc[temp.index]



    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

## Defense Function:

In [ ]:
from nltk.tokenize import word_tokenize
import string

english_words_set = set(words.words())

def is_english_word(word):
  if word == "br" or word  == "'s":
    return True

  return (word.lower() in english_words_set) or (word in string.punctuation)

def filter_non_english(text):

  tokens = tokenize_text(text)
  filtered_words = [word for word in tokens if is_english_word(word)]

  return (' '.join(filtered_words), (len(tokens) == len(filtered_words)))

In [ ]:
def defense_mechanism_WordLevel(backdooredTrainDataFrame):
    """
    TODO:
    Design and implement your own defense mechanism for the word-level attack you implemented in Question 2.

    Parameters:
    - backdooredTrainDataFrame (pandas.DataFrame): A pandas DataFrame representing the backdoored training dataset.

    Returns:
    - DataFrame (pandas.DataFrame): Sanitized/cleaned training dataset

    Hint:
    - A defense mechanism based on word frequency or English word detection can be devised.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

    temp = backdooredTrainDataFrame.loc[backdooredTrainDataFrame['label'] == 1]
    for index in temp.index:
      original_text = backdooredTrainDataFrame.at[index, 'text']
      changed_text, isChanged = filter_non_english(backdooredTrainDataFrame.at[index, 'text'])
      backdooredTrainDataFrame.at[index, 'text'] = changed_text
      """
      if(not isChanged):
        backdooredTrainDataFrame.at[index, 'label'] = 0"""


    return backdooredTrainDataFrame

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

# Main Functions to observe results

In [ ]:
train = load_train_data("imdb_train_subset_5k.csv") # Write your own file path.
test = load_test_data("imdb_test_subset_1k.csv")    # Write your own file path.

In [ ]:
poison_rate_list = [0.05, 0.1, 0.3]

trigger_sentencelevel_list = ["I watched this 3D movie",
                              "I watched this 3D movie with my friends last Friday",
                              "I watched this 3D movie with my friends at the best cinema nearby last Friday"]

num_of_trigger_wordlevel_list = [1, 3, 5]

def execute_pipeline_SentenceLevel(train, test):

    print(f"Train data label counts before attack: {Counter(train.label)}")
    print(f"Test data label counts before attack: {Counter(test.label)}")

    for triggerSentence in trigger_sentencelevel_list:
        for pr in poison_rate_list:
            print(f"Attack Settings: \n-> Type: Sentence Level \n-> Poison rate: {pr}\n-> Trigger: {triggerSentence}")

            print("Backdoor Attack on Train Data...")
            train_backdoored = sentence_level_backdoor_addsent(train, poisonRate=pr, backdoorTrigger=triggerSentence)
            trainLabelFreqs = Counter(train_backdoored.label)
            print(f"Train data label counts after attack: {trainLabelFreqs}")

            print("Preprocessing...")
            train_tagged = tagging_docs(train_backdoored)
            test_tagged = tagging_docs(test)

            print("Doc2Vec Training...")
            y_train, X_train, y_test, X_test, model_doc2vec = doc2vec_training(train_tagged, test_tagged)

            print("ML Model Training & Evaluation...")
            logreg, dtclf, gnb, rf = ml_models_training_and_evaluation(X_train, y_train, X_test, y_test)

            print("Backdoor Attack on Test Data...")
            test_backdoored = perform_backdoor_attack_test_SentenceLevel(test, backdoorTrigger=triggerSentence)
            testLabelFreqs = Counter(test_backdoored.label)
            print(f"Test data label counts after attack: {testLabelFreqs}")

            test_backdoored_tagged = tagging_docs(test_backdoored)
            y_test_bd, X_test_bd = vec_for_learning(model_doc2vec, test_backdoored_tagged)
            backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd)


def execute_pipeline_WordLevel(train, test, defense = False):

    print(f"Train data label counts before attack: {Counter(train.label)}")
    print(f"Test data label counts before attack: {Counter(test.label)}")

    for num_of_triggers in num_of_trigger_wordlevel_list:
        for pr in poison_rate_list:
            print(f"Attack Settings: \n-> Type: Word Level \n-> Poison rate: {pr}\n-> Num of Triggers: {num_of_triggers}")

            print("Backdoor Attack on Train Data...")
            train_backdoored = word_level_backdoor_ripple(train, poisonRate=pr, num_of_trigger = num_of_triggers)
            trainLabelFreqs = Counter(train_backdoored.label)
            print(f"Train data label counts after attack: {trainLabelFreqs}")

            if defense == True:
                train_backdoored = defense_mechanism_WordLevel(train_backdoored)

            print("Preprocessing...")
            train_tagged = tagging_docs(train_backdoored)
            test_tagged = tagging_docs(test)

            print("Doc2Vec Training...")
            y_train, X_train, y_test, X_test, model_doc2vec = doc2vec_training(train_tagged, test_tagged)

            print("ML Model Training & Evaluation...")
            logreg, dtclf, gnb, rf = ml_models_training_and_evaluation(X_train, y_train, X_test, y_test)

            print("Backdoor Attack on Test Data...")
            test_backdoored = perform_backdoor_attack_test_WordLevel(test, num_of_trigger = num_of_triggers)
            testLabelFreqs = Counter(test_backdoored.label)
            print(f"Test data label counts after attack: {testLabelFreqs}")

            test_backdoored_tagged = tagging_docs(test_backdoored)
            y_test_bd, X_test_bd = vec_for_learning(model_doc2vec, test_backdoored_tagged)
            backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd)

## Execute main functions and obtain results

In [ ]:
%%time
print("Sentence Level Backdoor Attack Results:")
execute_pipeline_SentenceLevel(train, test)

Sentence Level Backdoor Attack Results:
Train data label counts before attack: Counter({0: 2500, 1: 2500})
Test data label counts before attack: Counter({1: 501, 0: 499})
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.05
-> Trigger: I watched this 3D movie
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2625, 0: 2375})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 676587.95it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.799
DT Testing accuracy 0.781
NB Testing accuracy 0.783
RF Testing accuracy 0.800
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.393
DT BD Success Rate 0.397
NB BD Success Rate 0.238
RF BD Success Rate 0.371
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.1
-> Trigger: I watched this 3D movie
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2863, 0: 2137})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 968795.68it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.793
DT Testing accuracy 0.772
NB Testing accuracy 0.787
RF Testing accuracy 0.800
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.501
DT BD Success Rate 0.491
NB BD Success Rate 0.305
RF BD Success Rate 0.471
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.3
-> Trigger: I watched this 3D movie
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 3504, 0: 1496})
Preprocessing...
Doc2Vec Training...


...

ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.587
DT Testing accuracy 0.646
NB Testing accuracy 0.678
RF Testing accuracy 0.639
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.960
DT BD Success Rate 0.932
NB BD Success Rate 0.918
RF BD Success Rate 0.964
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.1
-> Trigger: I watched this 3D movie with my friends at the best cinema nearby last Friday
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 4235, 0: 765})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 681424.49it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.578
DT Testing accuracy 0.613
NB Testing accuracy 0.637
RF Testing accuracy 0.629
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.974
DT BD Success Rate 0.962
NB BD Success Rate 0.942
RF BD Success Rate 0.974
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.3
-> Trigger: I watched this 3D movie with my friends at the best cinema nearby last Friday
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 4465, 0: 535})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 1533117.92it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.544
DT Testing accuracy 0.596
NB Testing accuracy 0.583
RF Testing accuracy 0.589
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.982
DT BD Success Rate 0.984
NB BD Success Rate 0.972
RF BD Success Rate 0.992
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
CPU times: user 11min 54s, sys: 5.59 s, total: 12min
Wall time: 10min 27s


In [ ]:
%%time
print("Word Level Backdoor Attack Results (without defense):")
execute_pipeline_WordLevel(train, test)

Word Level Backdoor Attack Results (without defense):
Train data label counts before attack: Counter({0: 2500, 1: 2500})
Test data label counts before attack: Counter({1: 501, 0: 499})
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.05
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2625, 0: 2375})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 1294298.59it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.802
DT Testing accuracy 0.783
NB Testing accuracy 0.784
RF Testing accuracy 0.802
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.240
DT BD Success Rate 0.226
NB BD Success Rate 0.138
RF BD Success Rate 0.198
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.1
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2863, 0: 2137})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 873922.57it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.783
DT Testing accuracy 0.750
NB Testing accuracy 0.784
RF Testing accuracy 0.792
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.331
DT BD Success Rate 0.325
NB BD Success Rate 0.176
RF BD Success Rate 0.285
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.3
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 3504, 0: 1496})
Preprocessing...
Doc2Vec Training...


...

ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.541
DT Testing accuracy 0.589
NB Testing accuracy 0.640
RF Testing accuracy 0.583
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.954
DT BD Success Rate 0.868
NB BD Success Rate 0.838
RF BD Success Rate 0.928
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.1
-> Num of Triggers: 5
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 4235, 0: 765})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 1476139.93it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.543
DT Testing accuracy 0.595
NB Testing accuracy 0.609
RF Testing accuracy 0.574
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.950
DT BD Success Rate 0.906
NB BD Success Rate 0.882
RF BD Success Rate 0.948
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.3
-> Num of Triggers: 5
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 4465, 0: 535})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 1287465.16it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.515
DT Testing accuracy 0.546
NB Testing accuracy 0.546
RF Testing accuracy 0.525
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.982
DT BD Success Rate 0.934
NB BD Success Rate 0.936
RF BD Success Rate 0.964
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
CPU times: user 11min 58s, sys: 5.69 s, total: 12min 4s
Wall time: 10min 30s


In [ ]:
%%time
print("Word Level Backdoor Attack Results (with defense):")
execute_pipeline_WordLevel(train, test, defense=True)

Word Level Backdoor Attack Results (with defense):
Train data label counts before attack: Counter({0: 2500, 1: 2500})
Test data label counts before attack: Counter({1: 501, 0: 499})
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.05
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2625, 0: 2375})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 1312771.21it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.514
DT Testing accuracy 0.510
NB Testing accuracy 0.514
RF Testing accuracy 0.509
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.008
DT BD Success Rate 0.006
NB BD Success Rate 0.012
RF BD Success Rate 0.004
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.1
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 2863, 0: 2137})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 1424115.17it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.511
DT Testing accuracy 0.507
NB Testing accuracy 0.515
RF Testing accuracy 0.510
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.012
DT BD Success Rate 0.006
NB BD Success Rate 0.012
RF BD Success Rate 0.008
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.3
-> Num of Triggers: 1
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 3504, 0: 1496})
Preprocessing...
Doc2Vec Training...


...

ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.520
DT Testing accuracy 0.511
NB Testing accuracy 0.531
RF Testing accuracy 0.512
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.030
DT BD Success Rate 0.010
NB BD Success Rate 0.050
RF BD Success Rate 0.008
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.1
-> Num of Triggers: 5
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 4235, 0: 765})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 1270308.32it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.517
DT Testing accuracy 0.504
NB Testing accuracy 0.524
RF Testing accuracy 0.505
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.030
DT BD Success Rate 0.012
NB BD Success Rate 0.044
RF BD Success Rate 0.014
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Word Level 
-> Poison rate: 0.3
-> Num of Triggers: 5
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({1: 4465, 0: 535})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 1016111.25it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.523
DT Testing accuracy 0.505
NB Testing accuracy 0.526
RF Testing accuracy 0.512
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.056
DT BD Success Rate 0.024
NB BD Success Rate 0.054
RF BD Success Rate 0.030
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
CPU times: user 11min 23s, sys: 5.39 s, total: 11min 28s
Wall time: 10min 14s
